Integrantes  
Bryann Alfaro  
Raul Jimenez  
Donaldo Garcia

In [68]:
pip install scapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Parte 1 - Analisis de paquetes

 Analisis estadistico

In [69]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii 
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

REFERENCIA: JUPYTER BRINDADO EN CLASE

Capture 10 paquetes con la herramienta scapy, imprima el tipo de datos, la longitud y el
contenido del pcap

In [70]:
pcap = sniff(count=10) #captura de 10 paquetes

In [71]:
#Impresion de tipo de dato, longitud y contenido
print(type(pcap))
print(len(pcap))
for i in range(len(pcap)):
  print(pcap[i])
pcap[0]

<class 'scapy.plist.PacketList'>
10
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Ether / IP / TCP 172.28.0.12:8080 > 172.28.0.1:58154 PA / Raw
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 A
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Ether / IP / TCP 172.28.0.12:8080 > 172.28.0.1:58154 A
Ether / IP / TCP 172.28.0.12:8080 > 172.28.0.1:58154 PA / Raw
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 A
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Ether / IP / TCP 172.28.0.12:8080 > 172.28.0.1:58154 PA / Raw


<Ether  dst=02:42:ac:1c:00:0c src=02:42:c4:39:2f:0f type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=545 id=39032 flags=DF frag=0 ttl=64 proto=6 chksum=0x4819 src=172.28.0.1 dst=172.28.0.12 |<TCP  sport=58154 dport=8080 seq=2328216845 ack=3215407175 dataofs=8 reserved=0 flags=PA window=501 chksum=0x5a59 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (2209647178, 258351801))] |<Raw  load='GET /socket.io/?EIO=3&sid=arHfkLKDPJ-jKQUoAAAA&t=OTANQmm&transport=polling HTTP/1.1\r\nHost: colab.research.google.com\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0,gzip(gfe)\r\nAccept: */*\r\nAccept-Encoding: gzip, deflate, br,gzip(gfe)\r\nAccept-Language: es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3\r\nAlt-Used: colab.research.google.com\r\nReferer: https://colab.research.google.com/\r\nSec-Fetch-Dest: empty\r\nSec-Fetch-Mode: cors\r\nSec-Fetch-Site: same-origin\r\n\r\n' |>>>>

Para conocer un poco mas se puede hacer la exploracion

In [72]:
ethernet_frame = pcap[0]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload #recupera la informacion de la capa 4

print("Informacion del frame:")
print(ethernet_frame.summary())

print("Informacion del paquete:")
print(ip_packet.summary())

print("Informacion del segmento:")
print(segment.summary())

print("Data:")
print(data.summary())

ethernet_frame.show()
data.show()

Informacion del frame:
Ether / IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Informacion del paquete:
IP / TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Informacion del segmento:
TCP 172.28.0.1:58154 > 172.28.0.12:8080 PA / Raw
Data:
Raw
###[ Ethernet ]### 
  dst       = 02:42:ac:1c:00:0c
  src       = 02:42:c4:39:2f:0f
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 545
     id        = 39032
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0x4819
     src       = 172.28.0.1
     dst       = 172.28.0.12
     \options   \
###[ TCP ]### 
        sport     = 58154
        dport     = 8080
        seq       = 2328216845
        ack       = 3215407175
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 501
        chksum    = 0x5a59
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (2209647178, 25835180

In [73]:
pcap2 = rdpcap('analisis_paquetes.pcap')

In [74]:
print(len(pcap2))

62


In [75]:
pcap = pcap + pcap2

In [76]:
print(len(pcap))

72


Convierta el pcap a un DataFrame

In [77]:
# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

print(df.head(5))

  version ihl tos  len     id flags frag ttl proto chksum  ... dataofs  \
0       4   5   0  545  39032    DF    0  64     6  18457  ...       8   
1       4   5   0  713  14214    DF    0  64     6  43107  ...       8   
2       4   5   0   52  39033    DF    0  64     6  18949  ...       8   
3       4   5   0  665  39034    DF    0  64     6  18335  ...       8   
4       4   5   0  363  39035    DF    0  64     6  18636  ...       8   

  reserved flags window chksum urgptr options payload  \
0        0    PA    501  23129      0       3     493   
1        0    PA    501  23297      0       3     661   
2        0     A    501  22636      0       3       0   
3        0    PA    501  23249      0       3     613   
4        0    PA    501  22947      0       3     311   

                                         payload_raw  \
0  b'GET /socket.io/?EIO=3&sid=arHfkLKDPJ-jKQUoAA...   
1  b'HTTP/1.1 200 OK\r\nContent-Type: text/plain;...   
2                                           

Muestre los valores de las columnas: Src Addres, Dst Address, Src Port y Dst Port

In [78]:
df.columns

Index(['version', 'ihl', 'tos', 'len', 'id', 'flags', 'frag', 'ttl', 'proto',
       'chksum', 'src', 'dst', 'options', 'time', 'sport', 'dport', 'seq',
       'ack', 'dataofs', 'reserved', 'flags', 'window', 'chksum', 'urgptr',
       'options', 'payload', 'payload_raw', 'payload_hex'],
      dtype='object')

In [79]:
df['src']

0      172.28.0.1
1     172.28.0.12
2      172.28.0.1
3      172.28.0.1
4      172.28.0.1
         ...     
67    84.54.22.33
68     10.1.10.53
69    84.54.22.33
70     10.1.10.53
71    84.54.22.33
Name: src, Length: 72, dtype: object

In [80]:
df['src'].value_counts()

10.1.10.53     31
84.54.22.33    29
172.28.0.1      6
172.28.0.12     4
75.75.75.75     2
Name: src, dtype: int64

In [81]:
df['dst']

0     172.28.0.12
1      172.28.0.1
2     172.28.0.12
3     172.28.0.12
4     172.28.0.12
         ...     
67     10.1.10.53
68    84.54.22.33
69     10.1.10.53
70    84.54.22.33
71     10.1.10.53
Name: dst, Length: 72, dtype: object

In [82]:
df['dst'].value_counts()

10.1.10.53     31
84.54.22.33    29
172.28.0.12     6
172.28.0.1      4
75.75.75.75     2
Name: dst, dtype: int64

In [83]:
df['sport']

0     58154
1      8080
2     58154
3     58154
4     58154
      ...  
67       53
68       53
69       53
70       53
71       53
Name: sport, Length: 72, dtype: object

In [84]:
df['sport'].value_counts()

53       60
58154     6
8080      4
15812     1
23903     1
Name: sport, dtype: int64

In [85]:
df['dport']

0      8080
1     58154
2      8080
3      8080
4      8080
      ...  
67       53
68       53
69       53
70       53
71       53
Name: dport, Length: 72, dtype: object

In [86]:
df['dport'].value_counts()

53       60
8080      6
58154     4
15812     1
23903     1
Name: dport, dtype: int64

# Estadísticas

In [87]:
#Muestre cual es la IP origen más frecuente
source_ip = df['src'].value_counts(ascending=False).index[0]
source_ip

'10.1.10.53'

In [91]:
#Muestre cual es la IP destino más frecuente
dest_ip = df['dst'].value_counts(ascending=False).index[0]
dest_ip

'10.1.10.53'

In [89]:
#¿A qué IPs se comunica la IP del inciso a?
df[df['src']==source_ip][['src','dst']]

,src,dst
10,10.1.10.53,84.54.22.33
12,10.1.10.53,84.54.22.33
14,10.1.10.53,84.54.22.33
16,10.1.10.53,84.54.22.33
18,10.1.10.53,84.54.22.33
20,10.1.10.53,84.54.22.33
22,10.1.10.53,84.54.22.33
24,10.1.10.53,84.54.22.33
26,10.1.10.53,84.54.22.33
28,10.1.10.53,84.54.22.33


In [90]:
#¿A qué puertos destino se comunica la IP del inciso a?
df[df['src']==source_ip][['src','dport']]

,src,dport
10,10.1.10.53,53
12,10.1.10.53,53
14,10.1.10.53,53
16,10.1.10.53,53
18,10.1.10.53,53
20,10.1.10.53,53
22,10.1.10.53,53
24,10.1.10.53,53
26,10.1.10.53,53
28,10.1.10.53,53


In [92]:
#¿A qué puertos origen se comunica la IP del inciso b?
df[df['dst']==dest_ip][['dst','sport']]

,dst,sport
11,10.1.10.53,53
13,10.1.10.53,53
15,10.1.10.53,53
17,10.1.10.53,53
19,10.1.10.53,53
21,10.1.10.53,53
23,10.1.10.53,53
25,10.1.10.53,53
27,10.1.10.53,53
29,10.1.10.53,53
